# Boston Housing Case Study: Temporal vs Random Split Analysis

This notebook implements the case study requirements:
- Train a regression model using the first 70% of data (older houses)
- Evaluate on the last 30% (newer houses)
- Compare results to a random split baseline
- Analyze why performance may differ across the two splits
- Suggest ways to improve robustness to temporal changes

## Dataset Overview

The Boston Housing Dataset contains 506 samples with 13 features:
- **Features**: CRIM, ZN, INDUS, CHAS, NOX, RM, AGE, DIS, RAD, TAX, PTRATIO, B, LSTAT
- **Target**: MEDV (median house value in $1000s)

Note: Some MEDV values are censored at 50.0 (corresponding to $50,000)

## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
RANDOM_STATE = 42

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Load the dataset
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
data = pd.read_csv('../data/housing.csv', header=None, delimiter=r"\s+", names=column_names)

print(f"Dataset shape: {data.shape}")
print(f"Features: {list(data.columns[:-1])}")
print(f"Target: {data.columns[-1]}")
print("\nFirst few rows:")
data.head()

In [ ]:
# Basic dataset information
print("Dataset Info:")
print(f"Total samples: {len(data)}")
print(f"Features: {data.shape[1]-1}")
print(f"Target variable: {data.columns[-1]}")
print(f"\nTarget statistics:")
print(data['MEDV'].describe())
print(f"\nCensored values (MEDV >= 50): {(data['MEDV'] >= 50).sum()} ({(data['MEDV'] >= 50).sum()/len(data)*100:.1f}%)")

## 2. Quick EDA (Exploratory Data Analysis)

In [ ]:
# Check for missing values
print("Missing values:")
print(data.isnull().sum())

# Check data types
print("\nData types:")
print(data.dtypes)

In [ ]:
# Distribution of target variable
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
ax1.hist(data['MEDV'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
ax1.axvline(data['MEDV'].median(), color='red', linestyle='--', label=f'Median: {data["MEDV"].median():.1f}')
ax1.axvline(50, color='orange', linestyle='--', label='Censoring threshold (50)')
ax1.set_xlabel('MEDV (Median House Value in $1000s)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Target Variable (MEDV)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Box plot
ax2.boxplot(data['MEDV'])
ax2.set_ylabel('MEDV (Median House Value in $1000s)')
ax2.set_title('Box Plot of Target Variable')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 10))
correlation_matrix = data.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# Show top correlations with MEDV
medv_correlations = correlation_matrix['MEDV'].abs().sort_values(ascending=False)
print("Top correlations with MEDV:")
print(medv_correlations)

## 3. Data Preprocessing and Split Definitions

In [ ]:
# Create two versions of the dataset as per design
# Version 1: Keep all rows (including censored values)
data_all = data.copy()

# Version 2: Remove rows with MEDV >= 50.0 (censored values)
data_clean = data[data['MEDV'] < 50.0].copy()

print(f"Version 1 (Keep all): {data_all.shape[0]} samples")
print(f"Version 2 (Remove censored): {data_clean.shape[0]} samples")
print(f"Removed {data_all.shape[0] - data_clean.shape[0]} censored samples")

In [ ]:
# Define split functions
def create_chronological_split(data, train_ratio=0.7):
    """Create chronological split (first 70% vs last 30%)"""
    split_idx = int(len(data) * train_ratio)
    train_data = data.iloc[:split_idx]
    test_data = data.iloc[split_idx:]
    return train_data, test_data

def create_age_based_split(data, train_ratio=0.7):
    """Create AGE-descending split (top 70% AGE vs bottom 30% AGE)"""
    data_sorted = data.sort_values('AGE', ascending=False)
    split_idx = int(len(data_sorted) * train_ratio)
    train_data = data_sorted.iloc[:split_idx]
    test_data = data_sorted.iloc[split_idx:]
    return train_data, test_data

def create_random_split(data, train_ratio=0.7, random_state=RANDOM_STATE):
    """Create random split baseline"""
    train_data, test_data = train_test_split(data, train_size=train_ratio, 
                                            random_state=random_state, shuffle=True)
    return train_data, test_data

def prepare_features_target(data, scale_features=True):
    """Prepare features and target, optionally scale features"""
    X = data.iloc[:, :-1]  # All features except MEDV
    y = data.iloc[:, -1]   # MEDV
    
    if scale_features:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        return X_scaled, y, scaler
    else:
        return X, y, None

In [ ]:
# Create splits for both dataset versions
print("Creating splits for both dataset versions...")

# Version 1: Keep all data
print("\n=== Version 1: Keep All Data ===")
chrono_train_all, chrono_test_all = create_chronological_split(data_all)
age_train_all, age_test_all = create_age_based_split(data_all)
random_train_all, random_test_all = create_random_split(data_all)

print(f"Chronological split: {len(chrono_train_all)} train, {len(chrono_test_all)} test")
print(f"AGE-based split: {len(age_train_all)} train, {len(age_test_all)} test")
print(f"Random split: {len(random_train_all)} train, {len(random_test_all)} test")

# Version 2: Remove censored data
print("\n=== Version 2: Remove Censored Data ===")
chrono_train_clean, chrono_test_clean = create_chronological_split(data_clean)
age_train_clean, age_test_clean = create_age_based_split(data_clean)
random_train_clean, random_test_clean = create_random_split(data_clean)

print(f"Chronological split: {len(chrono_train_clean)} train, {len(chrono_test_clean)} test")
print(f"AGE-based split: {len(age_train_clean)} train, {len(age_test_clean)} test")
print(f"Random split: {len(random_train_clean)} train, {len(random_test_clean)} test")

## 4. Model Training and Evaluation

In [ ]:
# Define evaluation function
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
    """Train and evaluate a model, returning metrics"""
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return {
        'model_name': model_name,
        'rmse': rmse,
        'mae': mae,
        'r2': r2
    }

# Define models
models = {
    'Ridge': Ridge(alpha=1.0, random_state=RANDOM_STATE),
    'GradientBoosting': GradientBoostingRegressor(
        n_estimators=100, learning_rate=0.1, max_depth=3, 
        random_state=RANDOM_STATE
    )
}

In [ ]:
# Evaluate models on all splits and dataset versions
results = []

print("Evaluating models on different splits and dataset versions...")
print("=" * 80)

# Version 1: Keep all data
print("\nVERSION 1: Keep All Data (including censored)")
print("-" * 50)

for split_name, (train_data, test_data) in [
    ('Chronological', (chrono_train_all, chrono_test_all)),
    ('AGE-based', (age_train_all, age_test_all)),
    ('Random', (random_train_all, random_test_all))
]:
    print(f"\n{split_name} Split:")
    
    # Prepare features and target
    X_train, y_train, scaler = prepare_features_target(train_data, scale_features=True)
    X_test, y_test, _ = prepare_features_target(test_data, scale_features=False)
    
    # Scale test features using train scaler
    if scaler is not None:
        X_test = scaler.transform(X_test)
    
    # Evaluate each model
    for model_name, model in models.items():
        # For tree-based models, don't scale features
        if model_name == 'GradientBoosting':
            X_train_raw, y_train_raw, _ = prepare_features_target(train_data, scale_features=False)
            X_test_raw, y_test_raw, _ = prepare_features_target(test_data, scale_features=False)
            result = evaluate_model(model, X_train_raw, X_test_raw, y_train_raw, y_test_raw, model_name)
        else:
            result = evaluate_model(model, X_train, X_test, y_train, y_test, model_name)
        
        result['dataset_version'] = 'Keep All'
        result['split_type'] = split_name
        results.append(result)
        
        print(f"  {model_name}: RMSE={result['rmse']:.3f}, MAE={result['mae']:.3f}, R²={result['r2']:.3f}")

# Version 2: Remove censored data
print("\n\nVERSION 2: Remove Censored Data")
print("-" * 50)

for split_name, (train_data, test_data) in [
    ('Chronological', (chrono_train_clean, chrono_test_clean)),
    ('AGE-based', (age_train_clean, age_test_clean)),
    ('Random', (random_train_clean, random_test_clean))
]:
    print(f"\n{split_name} Split:")
    
    # Prepare features and target
    X_train, y_train, scaler = prepare_features_target(train_data, scale_features=True)
    X_test, y_test, _ = prepare_features_target(test_data, scale_features=False)
    
    # Scale test features using train scaler
    if scaler is not None:
        X_test = scaler.transform(X_test)
    
    # Evaluate each model
    for model_name, model in models.items():
        # For tree-based models, don't scale features
        if model_name == 'GradientBoosting':
            X_train_raw, y_train_raw, _ = prepare_features_target(train_data, scale_features=False)
            X_test_raw, y_test_raw, _ = prepare_features_target(test_data, scale_features=False)
            result = evaluate_model(model, X_train_raw, X_test_raw, y_train_raw, y_test_raw, model_name)
        else:
            result = evaluate_model(model, X_train, X_test, y_train, y_test, model_name)
        
        result['dataset_version'] = 'Remove Censored'
        result['split_type'] = split_name
        results.append(result)
        
        print(f"  {model_name}: RMSE={result['rmse']:.3f}, MAE={result['mae']:.3f}, R²={result['r2']:.3f}")

In [ ]:
# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(results)
print("Complete Results Summary:")
print("=" * 80)
print(results_df.round(3))

# Save results for later analysis
results_df.to_csv('model_evaluation_results.csv', index=False)
print("\nResults saved to 'model_evaluation_results.csv'")

## 5. Results Analysis and Visualization

In [ ]:
# Visualize results comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

# 1. RMSE comparison across splits and models
ax1 = axes[0]
pivot_rmse = results_df.pivot_table(values='rmse', index='split_type', 
                                    columns=['model_name', 'dataset_version'], aggfunc='mean')
pivot_rmse.plot(kind='bar', ax=ax1, color=['skyblue', 'lightcoral', 'lightgreen', 'orange'])
ax1.set_title('RMSE Comparison Across Splits and Models')
ax1.set_ylabel('RMSE')
ax1.legend(title='Model + Dataset Version', bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# 2. R² comparison
ax2 = axes[1]
pivot_r2 = results_df.pivot_table(values='r2', index='split_type', 
                                  columns=['model_name', 'dataset_version'], aggfunc='mean')
pivot_r2.plot(kind='bar', ax=ax2, color=['skyblue', 'lightcoral', 'lightgreen', 'orange'])
ax2.set_title('R² Comparison Across Splits and Models')
ax2.set_ylabel('R²')
ax2.legend(title='Model + Dataset Version', bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# 3. Model performance comparison (Ridge)
ax3 = axes[2]
ridge_results = results_df[results_df['model_name'] == 'Ridge']
ridge_pivot = ridge_results.pivot_table(values='rmse', index='split_type', 
                                       columns='dataset_version', aggfunc='mean')
ridge_pivot.plot(kind='bar', ax=ax3, color=['skyblue', 'lightcoral'])
ax3.set_title('Ridge Model: RMSE by Split Type and Dataset Version')
ax3.set_ylabel('RMSE')
ax3.legend(title='Dataset Version')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3)

# 4. Model performance comparison (GradientBoosting)
ax4 = axes[3]
gbr_results = results_df[results_df['model_name'] == 'GradientBoosting']
gbr_pivot = gbr_results.pivot_table(values='rmse', index='split_type', 
                                    columns='dataset_version'], aggfunc='mean')
gbr_pivot.plot(kind='bar', ax=ax4, color=['lightgreen', 'orange'])
ax4.set_title('GradientBoosting Model: RMSE by Split Type and Dataset Version')
ax4.set_ylabel('RMSE')
ax4.legend(title='Dataset Version')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Performance gap analysis
print("Performance Gap Analysis: Chronological vs Random Split")
print("=" * 60)

for dataset_version in ['Keep All', 'Remove Censored']:
    print(f"\n{dataset_version} Dataset:")
    
    for model_name in ['Ridge', 'GradientBoosting']:
        # Get results for chronological and random splits
        chrono_result = results_df[(results_df['dataset_version'] == dataset_version) & 
                                   (results_df['split_type'] == 'Chronological') & 
                                   (results_df['model_name'] == model_name)].iloc[0]
        
        random_result = results_df[(results_df['dataset_version'] == dataset_version) & 
                                   (results_df['split_type'] == 'Random') & 
                                   (results_df['model_name'] == model_name)].iloc[0]
        
        # Calculate performance gap
        rmse_gap = chrono_result['rmse'] - random_result['rmse']
        r2_gap = random_result['r2'] - chrono_result['r2']
        
        print(f"  {model_name}:")
        print(f"    RMSE gap (Chrono - Random): {rmse_gap:+.3f}")
        print(f"    R² gap (Random - Chrono): {r2_gap:+.3f}")
        
        if rmse_gap > 0:
            print(f"    → Chronological split performs WORSE by {rmse_gap:.3f} RMSE")
        else:
            print(f"    → Chronological split performs BETTER by {abs(rmse_gap):.3f} RMSE")

## 6. Drift and Error Analysis

In [ ]:
# Analyze feature distribution shifts between train and test sets
print("Feature Distribution Shift Analysis: Chronological Split")
print("=" * 60)

# Focus on key features as per design
key_features = ['LSTAT', 'RM', 'NOX', 'DIS', 'RAD', 'TAX']

for dataset_version, (train_data, test_data) in [
    ('Keep All', (chrono_train_all, chrono_test_all)),
    ('Remove Censored', (chrono_train_clean, chrono_test_clean))
]:
    print(f"\n{dataset_version} Dataset:")
    
    for feature in key_features:
        train_mean = train_data[feature].mean()
        test_mean = test_data[feature].mean()
        train_std = train_data[feature].std()
        test_std = test_data[feature].std()
        
        mean_shift = test_mean - train_mean
        std_shift = test_std - train_std
        
        print(f"  {feature}:")
        print(f"    Mean: {train_mean:.3f} → {test_mean:.3f} (shift: {mean_shift:+.3f})")
        print(f"    Std:  {train_std:.3f} → {test_std:.3f} (shift: {std_shift:+.3f})")
        
        # Statistical significance test
        _, p_value = stats.ks_2samp(train_data[feature], test_data[feature])
        print(f"    KS test p-value: {p_value:.4f} {'(significant)' if p_value < 0.05 else '(not significant)'}")

In [ ]:
# Visualize distribution shifts for key features
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, feature in enumerate(key_features):
    ax = axes[i]
    
    # Plot distributions for both dataset versions
    ax.hist(chrono_train_all[feature], bins=20, alpha=0.6, label='Train (Keep All)', 
             color='skyblue', density=True)
    ax.hist(chrono_test_all[feature], bins=20, alpha=0.6, label='Test (Keep All)', 
             color='lightcoral', density=True)
    
    ax.set_title(f'{feature} Distribution Shift')
    ax.set_xlabel(feature)
    ax.set_ylabel('Density')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Model explainability analysis
print("Model Explainability Analysis")
print("=" * 40)

# Ridge model coefficients (using chronological split, keep all data)
X_train, y_train, scaler = prepare_features_target(chrono_train_all, scale_features=True)
ridge_model = Ridge(alpha=1.0, random_state=RANDOM_STATE)
ridge_model.fit(X_train, y_train)

# Get feature names
feature_names = data.columns[:-1]

# Plot Ridge coefficients
plt.figure(figsize=(12, 6))
coefficients = ridge_model.coef_
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
}).sort_values('Coefficient', key=abs, ascending=False)

plt.barh(range(len(feature_importance)), feature_importance['Coefficient'], 
          color=['red' if x < 0 else 'blue' for x in feature_importance['Coefficient']])
plt.yticks(range(len(feature_importance)), feature_importance['Feature'])
plt.xlabel('Standardized Coefficient')
plt.title('Ridge Model: Feature Coefficients (Chronological Split, Keep All Data)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Top features by absolute coefficient magnitude:")
print(feature_importance.round(3))

In [ ]:
# GradientBoosting feature importance
X_train_raw, y_train_raw, _ = prepare_features_target(chrono_train_all, scale_features=False)
gbr_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, 
                                      random_state=RANDOM_STATE)
gbr_model.fit(X_train_raw, y_train_raw)

# Plot feature importance
plt.figure(figsize=(12, 6))
feature_importance_gbr = pd.DataFrame({
    'Feature': feature_names,
    'Importance': gbr_model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.barh(range(len(feature_importance_gbr)), feature_importance_gbr['Importance'], 
          color='lightgreen')
plt.yticks(range(len(feature_importance_gbr)), feature_importance_gbr['Feature'])
plt.xlabel('Feature Importance')
plt.title('GradientBoosting Model: Feature Importance (Chronological Split, Keep All Data)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Top features by importance:")
print(feature_importance_gbr.round(3))

## 7. Findings and Recommendations

In [ ]:
# Summary of key findings
print("KEY FINDINGS AND RECOMMENDATIONS")
print("=" * 50)

# 1. Performance comparison summary
print("\n1. PERFORMANCE COMPARISON:")
print("   - Chronological split performance vs Random split baseline")
print("   - Impact of censored data removal")
print("   - Model robustness across different splits")

# 2. Drift analysis summary
print("\n2. TEMPORAL DRIFT ANALYSIS:")
print("   - Feature distribution shifts between train and test sets")
print("   - Statistical significance of observed shifts")
print("   - Key features contributing to drift")

# 3. Model explainability
print("\n3. MODEL EXPLAINABILITY:")
print("   - Ridge model: Standardized coefficients")
print("   - GradientBoosting: Feature importance rankings")
print("   - Feature contribution to predictions")

# 4. Recommendations
print("\n4. ROBUSTNESS RECOMMENDATIONS:")
print("   - Regularized linear models for stability")
print("   - Periodic model retraining")
print("   - Monitor covariate drift on key features")
print("   - Time-aware validation strategies")
print("   - Ensemble methods for robustness")

# 5. Next steps
print("\n5. NEXT STEPS:")
print("   - Implement rolling window validation")
print("   - Test additional robustness techniques")
print("   - Deploy monitoring systems")
print("   - Establish retraining protocols")

In [ ]:
# Create a comprehensive summary table
print("COMPREHENSIVE RESULTS SUMMARY")
print("=" * 50)

# Best performing configuration for each split type
best_configs = results_df.loc[results_df.groupby('split_type')['rmse'].idxmin()]
print("\nBest Configuration for Each Split Type:")
print(best_configs[['split_type', 'model_name', 'dataset_version', 'rmse', 'mae', 'r2']].round(3))

# Performance ranking
print("\nOverall Performance Ranking (by RMSE):")
performance_ranking = results_df.sort_values('rmse')[['split_type', 'model_name', 'dataset_version', 'rmse', 'mae', 'r2']]
print(performance_ranking.round(3))

# Save comprehensive results
performance_ranking.to_csv('performance_ranking.csv', index=False)
print("\nPerformance ranking saved to 'performance_ranking.csv'")

# Final recommendations
print("\nFINAL RECOMMENDATIONS:")
print("1. Use Ridge regression with chronological split for temporal analysis")
print("2. Monitor feature drift on LSTAT, RM, NOX, DIS, RAD, TAX")
print("3. Implement regular retraining schedule")
print("4. Consider ensemble methods for improved robustness")
print("5. Establish drift detection and alerting systems")